# Einführung in Informatik in Kultur und Gesundheit
## Gesundheitsinformatik Teil 4

Die beiden letzten Seminare haben wir uns damit beschäftigt, wie man FASTA-Dateien einlesen kann und welche herausforderungen dabei bestehen. Noch mal zur Erinnerung. FASTA-Dateien werden genutzt um Genomische oder Proteinsequenzen abzuspeichern. Daher muss man immer darauf achten was in der Datei tatsächlich enthalten ist.

Im Seminar haben wir auch besprochen, wie das NGS (Shortread Sequencing) funktioniert. Daher wollen wir uns nun mit den daraus resultierenden Daten etwas beschäfgigen. Zunächst kümmern wir uns um die heutigen imports und anschließend laden wir uns wieder die Daten herunter. Dazu nutzen wir wieder den Code aus dem 2. Seminar. 

In [ ]:
import requests # Zum herunterladen der Daten
import seaborn as sns # Zum Visualisieren
import os       # Zum erstellen und löschen von Verzeichnissen
import pandas
import matplotlib.pyplot as plt
import math

### Daten Downloader:

In [ ]:
directory = "EKG_S4"
if not os.path.isdir(directory):
    os.mkdir("EKG_S4")
# Inhalt des neuen Verzeichnisses Anzeigen    
print(os.listdir("EKG_S4"))


def downloadData (url,dateipfad):
    with open(dateipfad,"w") as datei:
        response = requests.get(url)
        print("Status Code des HTTP-Requests:",response.status_code)
        if response.status_code ==200:
            datei.write(response.text)


### Die Daten

In [ ]:
daten = "EKG_S4/Sample1.fastq"
downloadData("https://raw.githubusercontent.com/htw-wulf/EKG_S4/main/daten/Sample1.fastq","EKG_S4/Sample1.fastq")

## Das FASTQ-Format

Ähnlich wie das FASTA-Format ist das FASTQ-Format ein Plaintext Format. Dieses mal können jedoch nur Daten genomischen Ursprungs enthalten sein. Proteindaten können wir also ausschließen. Im Fall vom NGS befinden sich auch ausschließlich Sequenzdaten aus dem Alphabet "A","C","G","T","N" in den Daten und die Sequenzen sind recht kurz <= 310BP.

Ein einzellner Eintrag besteht immer aus 4 Zeilen. Tut er das nicht ist der Eintrag unvollständig. Dabei sind die Inhalte der Zeilen wie folgt aufgebaut:

1. Header
2. Sequenzdaten
3. Kommentar
4. Sequenzierqualitäten codiert als ASCII Symbole

### Header
[Detail Beschreibung](https://help.basespace.illumina.com/files-used-by-basespace/fastq-files) und [Hier](https://knowledge.illumina.com/software/general/software-general-reference_material-list/000002211)

### Aufbau eines Einzelleintrags


***Beispiel-Header:*** `@<instrument>:<run number>:<flowcell ID>:<lane>:<tile>:<x-pos>:<y-pos>(/<read>)` (in Klammern ist optional)

***Beispiel-Sequenz:*** `ATGCTTAGCATATCGTTTAGGCN`

***Beispiel-Kommentar:*** `+`

***Beispiel-Sequencing Qualities:*** `EAAAEADDDFFADFASDFASDF+`



***Aufgabe 1***: Definieren Sie eine Klasse `FASTQ_ENTRY` die die 4 Zeilen eines FASTQ-Eintrags als Einzellargumente bekommt. Dabei soll der Header bei der Initialisierung  in seine einzellnen erforderlichen Felder aufgeteilt werden. Zudem soll die BuildIn  `len()` Funktion die Länge der Sequenz zurück geben. 



In [ ]:
class FASTQ_ENTRY:
    def __init__(self,xxx):
        pass
    
    def __len__(self):
        pass
        

***Aufgabe 2:*** 
Schreiben Sie eine Funktion `readfastq(path)` die einen Dateipfad als Eingabe bekommt und eine Liste an FASTQ_ENTRYS zurück gibt.

In [ ]:
def readfastq(path):
    pass
sample1 = readfastq(daten)
display(sample1[:5])

***Aufgabe 3:*** Schreiben Sie nun eine Funktion `basic_stats(FASTQListe)`die auf Basis der Liste an FASTQ Objekten folgende Statistiken erstellt und als Tuple in der Reihenfolge zurück gibt:
- Anzahl der Reads
- Anzahl Sequenzierter Basen
- Kleinste Read-Länge
- Größte Read-Länge


In [ ]:
def basic_stats(FASTQListe):
    pass

print("# Reads: %d\n# Sequenzierte Basen: %d\nkürzester Read: %d\nlängster Read: %d"%basic_stats(sample1))

## Phredscores
Phred quality scores `Q` sind logarithmische Darstellungen der base-calling error Wahrscheinlichkeiten P  und *so* definiert:

`Q = − 10 *log_10(P)`

Phred Quality Score |	Probability of incorrect base call |	Base call accuracy
--------------------|--------------------------------------|-----------------------
10 |	1 in 10 	|90%
20 |	1 in 100 	|99%
30 |	1 in 1000 	|99.9%
40 |	1 in 10,000 |	99.99%
50 |	1 in 100,000 |	99.999%
60 |	1 in 1,000,000 | 	99.9999% 


Um diese Phredscores besser abspeichern zu können werden diese im ASCII Raum der Duckbaren Zeichen abgespeichert char(33 + phredscore). Hier einmal zum Testen . Genauigkeit zu Phred zum berechnen:

In [ ]:
def prob2phred(prob):
    return round(-10* math.log10(prob))

fehler_wahrscheinlichkeit = [1,0.9,0.5,0.1,0.01,0.00001]

for p in fehler_wahrscheinlichkeit:
    print(p,"->"  , prob2phred(p))

***Aufgabe 4:*** Schreiben Sie eine Funktion  `qualstring2phredlist(qual)`, die die Qualitätsstrings der FASTQs in Integer Listen wandelt und schreiben Sie eine Funktion `phredlist2qualstring(phredlist)`, die `qualstring2phredlist(qual)` umkehren kann.

In [ ]:
def qualstring2phredlist(qual):
    pass

def phredlist2qualstring(phredlist):
    pass

print(qualstring2phredlist("!$~"),[0, 3, 93])
print(phredlist2qualstring([0, 3, 93]),"!$~")
print(phredlist2qualstring(qualstring2phredlist("!!$&AF")) == "!!$&AF")
print(qualstring2phredlist(phredlist2qualstring([0,8,15]))==[0,8,15])

***Aufgabe 5*** 
Schreiben Sie eine Funktion `fastqlist2positional_qualities (fastqlist)` die für jeden read in einer FASTQ-Liste die Position und die einzelne Qualitätfür alle Reads als pandas Dataframe ausgibt.

In [ ]:
def fastqlist2positional_qualities (fastqlist):
        pass
df = fastqlist2positional_qualities(sample1)
display(df)

***Aufgabe 6***
Plotten Sie in einem Boxplot die Qualitäten der Reads und Teilen Sie dabei die einzelnen Positionen auf der X-Achse auf. Was sehen Sie?

In [ ]:
sns.set(rc={'figure.figsize':(60,10)})


## ASCII TABELLE

int | Phred |char
----|--|---
0  |   | 
1  |   | 
2  |   | 
3  |   | 
4  |   | 
5  |   | 
6  |   | 
7  |   | 
8  |   |
9  |   | 	
10  |   | 
11  |   | 
12  |   | 
13  |   | 
14  |   | 
15  |   | 
16  |   | 
17  |   | 
18  |   | 
19  |   | 
20  |   | 
21  |   | 
22  |   | 
23  |   | 
24  |   | 
25  |   | 
26  |   | 
27  |   | 
28  |   | 
29  |   | 
30  |   | 
31  |   | 
32  |   |  
33  |  0 | !
34  |  1 | "
35  |  2 | #
36  |  3 | $
37  |  4 | %
38  |  5 | &
39  |  6 | '
40  |  7 | (
41  |  8 | )
42  |  9 | *
43  |  10 | +
44  |  11 | ,
45  |  12 | -
46  |  13 | .
47  |  14 | /
48  |  15 | 0
49  |  16 | 1
50  |  17 | 2
51  |  18 | 3
52  |  19 | 4
53  |  20 | 5
54  |  21 | 6
55  |  22 | 7
56  |  23 | 8
57  |  24 | 9
58  |  25 | :
59  |  26 | ;
60  |  27 | <
61  |  28 | =
62  |  29 | >
63  |  30 | ?
64  |  31 | @
65  |  32 | A
66  |  33 | B
67  |  34 | C
68  |  35 | D
69  |  36 | E
70  |  37 | F
71  |  38 | G
72  |  39 | H
73  |  40 | I
74  |  41 | J
75  |  42 | K
76  |  43 | L
77  |  44 | M
78  |  45 | N
79  |  46 | O
80  |  47 | P
81  |  48 | Q
82  |  49 | R
83  |  50 | S
84  |  51 | T
85  |  52 | U
86  |  53 | V
87  |  54 | W
88  |  55 | X
89  |  56 | Y
90  |  57 | Z
91  |  58 | [
92  |  59 | \
93  |  60 | ]
94  |  61 | ^
95  |  62 | _
96  |  63 | `
97  |  64 | a
98  |  65 | b
99  |  66 | c
100  |  67 | d
101  |  68 | e
102  |  69 | f
103  |  70 | g
104  |  71 | h
105  |  72 | i
106  |  73 | j
107  |  74 | k
108  |  75 | l
109  |  76 | m
110  |  77 | n
111  |  78 | o
112  |  79 | p
113  |  80 | q
114  |  81 | r
115  |  82 | s
116  |  83 | t
117  |  84 | u
118  |  85 | v
119  |  86 | w
120  |  87 | x
121  |  88 | y
122  |  89 | z
123  |  90 | {
124  |  91 | |
125  |  92 | }
126  |  93 | ~
127  |  94 | 

In [ ]:
for x in range(128):
    print(x," | ", (x)-33 if x>=33 else "", "|", chr(x))